Himani Parikh - 1322085

CSCI 636 - Big Data Analytics

Hands on 8.1 : Spark SQL

***Install pyspark***

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=f9fcbffa9712c902453bc4b0dce831f6081c218365848f6823fa8baca8cb3848
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


***Initiate the spark context***

In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

spark = SparkSession\
        .builder\
        .master('local[*]')\
        .getOrCreate()
sc = spark.sparkContext

***Mount Google drive***

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# change the directory to big-data-colab folder
%cd /content/drive/My Drive/Colab Notebooks/BigDataColabHimaniParikh

#pwd
!ls

/content/drive/My Drive/Colab Notebooks/BigDataColabHimaniParikh
2.DataCopyHimani.ipynb		   Data		  out_wordmedian
2_WordCountHimani.ipynb		   Data_join	  SparkOutput
3.1_WordGrepHimani.ipynb	   Data_join2	  SqlDataSet
3.2_WordMedianHimani.ipynb	   out		  testfile1
4.MR-DataJoinHimani.ipynb	   out_Datajoin   testfile2
4.MR-WordCountHimani.ipynb	   outDatajoin2   wordcount_mapper.py
5.MR-DataJoin2Himani.ipynb	   out_grep	  wordcount_num0_output.txt
6.MR-WordCountReducerHimani.ipynb  out_testfile   wordcount_num2_output.txt
7.Spark-WordCountHimani.ipynb	   out_testfile1  wordcount_reducer.py
BigDataHimani.ipynb		   out_testfile2


***Connect to SQL Table***

In [5]:
from pyspark.sql import SQLContext
sqlsc = SQLContext(sc)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [8]:
df = sqlsc.read.load('/content/drive/My Drive/Colab Notebooks/BigDataColabHimaniParikh/SqlDataSet/game-clicks.csv',
                     format='com.databricks.spark.csv',
                     header='true',
                     inferSchema='true')

***View Spark DataFrame schema and count rows***

In [9]:
df.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- clickId: integer (nullable = true)
 |-- userId: integer (nullable = true)
 |-- userSessionId: integer (nullable = true)
 |-- isHit: integer (nullable = true)
 |-- teamId: integer (nullable = true)
 |-- teamLevel: integer (nullable = true)



In [10]:
df.count()

755806

***View contents of DataFrame.***

In [11]:
df.show(5)

+-------------------+-------+------+-------------+-----+------+---------+
|          timestamp|clickId|userId|userSessionId|isHit|teamId|teamLevel|
+-------------------+-------+------+-------------+-----+------+---------+
|2016-05-26 15:06:55|    105|  1038|         5916|    0|    25|        1|
|2016-05-26 15:07:09|    154|  1099|         5898|    0|    44|        1|
|2016-05-26 15:07:14|    229|   899|         5757|    0|    71|        1|
|2016-05-26 15:07:14|    322|  2197|         5854|    0|    99|        1|
|2016-05-26 15:07:20|     22|  1362|         5739|    0|    13|        1|
+-------------------+-------+------+-------------+-----+------+---------+
only showing top 5 rows



***Filter columns in DataFrame.***

In [12]:
df.select("userid", "teamlevel").show(5)

+------+---------+
|userid|teamlevel|
+------+---------+
|  1038|        1|
|  1099|        1|
|   899|        1|
|  2197|        1|
|  1362|        1|
+------+---------+
only showing top 5 rows



***Filter rows based on criteria.***

In [13]:
df.filter(df["teamlevel"] > 1).select("userid", "teamlevel").show(5)

+------+---------+
|userid|teamlevel|
+------+---------+
|  1513|        2|
|   868|        2|
|  1453|        2|
|  1282|        2|
|  1473|        2|
+------+---------+
only showing top 5 rows



***Group by a column and count.***

In [14]:
df.groupBy("ishit").count().show()

+-----+------+
|ishit| count|
+-----+------+
|    1| 83383|
|    0|672423|
+-----+------+



***Calculate average and sum***

In [16]:
from pyspark.sql.functions import*

df.select(mean('ishit'), sum('ishit')).show()

+------------------+----------+
|        avg(ishit)|sum(ishit)|
+------------------+----------+
|0.1103232840173272|     83383|
+------------------+----------+



***Join two DataFrames.***

In [17]:
df2 = sqlsc.read.load('/content/drive/My Drive/Colab Notebooks/BigDataColabHimaniParikh/SqlDataSet/ad-clicks.csv',
                     format='com.databricks.spark.csv',
                     header='true',
                     inferSchema='true')

In [18]:
df2.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- txId: integer (nullable = true)
 |-- userSessionId: integer (nullable = true)
 |-- teamId: integer (nullable = true)
 |-- userId: integer (nullable = true)
 |-- adId: integer (nullable = true)
 |-- adCategory: string (nullable = true)



In [19]:
merge = df.join(df2, 'userid')

***Let's view the schema of merge:***

In [20]:
merge.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- clickId: integer (nullable = true)
 |-- userSessionId: integer (nullable = true)
 |-- isHit: integer (nullable = true)
 |-- teamId: integer (nullable = true)
 |-- teamLevel: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- txId: integer (nullable = true)
 |-- userSessionId: integer (nullable = true)
 |-- teamId: integer (nullable = true)
 |-- adId: integer (nullable = true)
 |-- adCategory: string (nullable = true)



In [21]:
merge.show(5)

+------+-------------------+-------+-------------+-----+------+---------+-------------------+-----+-------------+------+----+-----------+
|userId|          timestamp|clickId|userSessionId|isHit|teamId|teamLevel|          timestamp| txId|userSessionId|teamId|adId| adCategory|
+------+-------------------+-------+-------------+-----+------+---------+-------------------+-----+-------------+------+----+-----------+
|  1362|2016-05-26 15:07:20|     22|         5739|    0|    13|        1|2016-06-16 10:21:01|39733|        34223|    13|   1|     sports|
|  1362|2016-05-26 15:07:20|     22|         5739|    0|    13|        1|2016-06-15 23:52:15|38854|        34223|    13|   3|electronics|
|  1362|2016-05-26 15:07:20|     22|         5739|    0|    13|        1|2016-06-15 12:23:31|37940|        34223|    13|  15|     sports|
|  1362|2016-05-26 15:07:20|     22|         5739|    0|    13|        1|2016-06-13 00:12:01|32627|        26427|    13|  14|    fashion|
|  1362|2016-05-26 15:07:20|     2